# 树

## 定义问题

当我们说“树形结构”的时候，那是从自然界的大树上得到的启发，从根开始不断展开的枝丫，代表了从单一到复杂的一层层展开，在现实世界充满了这样的事物：国家的行政区划（以及任何组织的架构）、电商网站的商品分类、论坛里的板块等等。

<img src="assets/tree-graph.png" width="700">

> 虽然自然界里树是根在下面而向上分支，但是我们在处理树形结构时，如图这样的从上向下分支更容易画也更容易看。

解决问题的第一步是先要清晰明确地定义问题，我们来尝试把我们对树的直觉印象书写成尽可能清晰明确的定义和表述。在一个树形结构里：
* 树（*tree*）由节点（*node*）和连接节点的边（*edge*）组成；
* 总有一个节点是分支的起点，它分出了所有其他的节点，这个节点叫根节点（*root node*）；
* 一个节点分支出来的节点叫它的“子节点（*child nodes*）”，它是其子节点的“父节点（*parent node*）”；上图中节点 A 的父节点是根节点，子节点是 B 和 C，而节点 C 的父节点是 A，子节点是 D 和 E；节点 D 的父节点是 C，而它没有子节点；
* 拥有共同父节点的两个节点互为“兄弟姐妹（*sibling nodes*）”；比如上图中 B 和 C，D 和 E；
* 没有子节点的节点也叫“叶子节点（*leaf node*）”；比如上图中的 D 和 E；
* 一个节点的父节点，以及父节点的父节点…直至根节点，都是这个节点的“祖先节点（*ancestor nodes*）”；比如上图中 E 的祖先节点包括 C、A 和 根节点；
* 一个节点的所有子节点，以及所有子节点的子节点…直至叶子节点，都是这个节点的“后代节点（*descendant nodes*）”；
* 一个节点 X 和它所有后代节点、以及这些节点之间的边，也组成一棵树，叫做原数的一棵（以 X 为根的）“子树（*sub-tree*）”；
* 根节点没有父节点也没有祖先节点；
* 除了根节点以外的任何节点，有且只有 1 个父节点，有至少 1 个祖先节点；
* 任何节点都可以有 0、1 或者多个子节点，可以有  0、1 或者多个后代节点；
* 一个节点和它的任一祖先或者后代节点之间，一定存在一条由边首尾连接组成的路径（*path*），比如上图中根节点是 E 的祖先，它们之间的路径是 `root -> A -> C -> E`；这个路径就像是两个节点之间的亲属关系链；
* 两个节点之间 *path* 的长度，就是它由几条边组成，决定了这两个节点之间隔了多少“代”，也叫节点之间的“距离（*distance*）”；
* 一个节点和根节点之间的 *distance* 经常有特别的含义，相当于该节点在树的“第几层级（*tier*）”；比如在行政区划里哪些是一级节点（省、直辖市、自治区）哪些是二级节点等；
* 有些树里节点的子节点是有顺序的，叫做“有序树（*ordered tree*）”；如果我们不特别指出，那就是不考虑这种顺序概念。

上面这种“把直觉规则化”的过程，对后面设计解决方案是至关重要的，大家可以自己尝试，多多体会。

> 事实上在数学图论里有对树的[更严谨更数学化的定义](https://en.wikipedia.org/wiki/Tree_(graph_theory))，不过上面的表述对目前的我们来说基本够用了。

## 分析操作场景

可以看到树和我们前面讲的所有数据结构都不一样，没办法用一种“线性（*linear*）”结构来表达，如何在计算机中实现一个树形数据结构？面对这样的问题，我们应该如何思考呢？

一个优秀的起点是问题解决的一半。设计数据结构的起点是：**思考我们会怎么操作和使用这个数据结构**。

假定我们已经有一个树形数据结构，我们可能会有这些操作场景（可以尝试用行政区划或者论坛版块等熟悉的实例来帮助思考）：
* 查找和遍历：
  * 给定一个 *node* 找出它的 *parent* ✪
  * 给定一个 *node* 找出它所有的 *children* ✪
  * 给定一个 *node* 找出它某个特定 *child*
  * 给定一个 *node* 找出它所有的 *siblings*
  * 给定一个 *node* 遍历其 *sub-tree* 即找出它的所有 *descendants* ✪
  * 给定 *node* *A* 和 *B*，找到 *A* 和 *B* 之间的 *path* 和 *distance*
  * 给定一个 *node* 确定它的 *tier*
* 编辑
  * 给一个 *node* 增加一个 *child* ✪
  * 删除一个 *node* ✪ -> 新一步思考：这意味着什么？删除整个子树还是？
  * 修改一个 *node* 的 *parent* ✪

差不多就这些，其中标记了 ✪ 号的是感觉特别重要和基础的操作。

通过形象化的图示、对基本概念的定义和表述、对可能操作场景的罗列，已经增加了很多我们对问题理解的深度和全面度，下来我们可以尝试做出一些初步设计判断了。

## 设计初步方案

从前面的分析，我们可以有这么一些收获：
* 树中节点之间的父子关系是核心和本质的内容；
* 表达这种父子关系的关键是节点，节点应保存父节点和子节点相关信息；
* 树的结构上带有显著的“递归”特点，可以有助于我们的设计。

如果你不记得“递归（*recursion*，*recursive*）”是怎么回事了，可以温习[递归函数](p2-4-recursion.ipynb)一章。

树也是递归的一个典型例子，因为一棵树可以看做由**根节点、根节点的子节点和以这些子节点为根的子树**合起来组成的，如果我们想实现对树进行操作的函数 `f()`，那么我们可以让这个函数接受一个树的根节点作为输入参数，这个函数大致上会是这个样子：

```python
def f(root):
    # 对 root 做一些操作
    # 然后取出 root 的所有子节点，对其中每个子节点调用 f() 本身
    for node in root.children:
        f(node)
```

也就是说，我们只要对某个节点做操作，然后获取这个节点的所有子节点，递归调用自己，就能对整个树做操作了。

在树形数据结构中，“获取一个节点的所有子节点”是至关重要的操作。

据此我们可以做出这么一个初步的设计：

In [ ]:
# 请在这里输入代码
class TreeNode:
    def __init__(self, name='root', data=None, parent=None, children=None):
        self.name = self.name
        self.data = data
        
        if parent:
            # 确认parent参数是TreeNode类型
            assert isinstance(parent, TreeNode)
            parent.add_child(self)
        self.parent = parent
        
        self.children = []
        if children:
            for child in children:
                self.add_child(child)
                
    def add_child(self, node):
        # 1. 确认node参数是TreeNode类型
        # 2. 将要加入的子节点的parent属性设为自己
        # 3. 然后将其加入children列表
        assert isinstance(node, TreeNode)
        node.parent =self
        self.children.append(node)

我们设计的核心数据结构是表示树节点的自定义类型 TreeNode，这个类型的对象有四个实例变量（属性）：
* *name*：节点的名字，最好能唯一标识出一个节点
* *data*：节点相关的任何数据，可以是任何数据类型
* *parent*：节点的父节点，如果没有父节点就是 None
* *children*：节点所有子节点组成的一个列表

这里面 *parent* 和 *children* 里的元素都必须是 TreeNode 类型的对象，我们在处理这两个属性时要先确认这一点，在上面的代码中我们用 Python 的 `assert` 语句和 `isinstance()` 函数来实现：
* `assert` 关键字后面的表达式必须返回 True，否则程序将抛出 `AssertionError` 异常后终止；
* `isinstance(obj, type)` 之前我们就介绍过，它接受两个参数，第一个参数是一个对象，而第二个参数是一个类型，函数判断第一个参数是不是第二个参数指明的类型，如果是返回 `True`，否则返回 `False`。

如上定义的 `TreeNode` 类，它实例化出来的对象 `node` 具备如下的能力：
* 很容易取得其父节点 *parent*；
* 很容易取得其所有子节点 *children*；
* 已经实现了增加子节点的操作。

最有意思的是，TreeNode 类型实际上也代表了树本身，因为一个节点加上它所有子节点，本来就是一棵树嘛！

在这个类型定义的基础上，我们可以实现前一节列出的所有操作，你可以把这作为练习动手试一试。

> 在实际项目中，我们并不需要定义树的数据结构，因为有优秀的第三方实现可用，比如 Python 非常棒的第三方库 [anytree](https://anytree.readthedocs.io/en/latest/#)，你可以试试，看和你自己实现的有什么区别。

## 小结

这一章介绍非常常见的树形逻辑怎么实现。没有很多代码，主要讲的是思维方法，请你读完一遍之后尝试自己从头思考和建立一个基本的树的数据结构，如果遇到问题就再读一遍。

In [ ]:
lst = [1, 2, 3, 4, 5]

In [ ]:
lst

In [ ]:
lst.reverse()

In [ ]:
print(lst.reverse())

In [ ]:
lst

In [ ]:
lst2 = lst.reverse()

In [ ]:
lst2

由于对程序某一个细节的运行机制，某一个函数的用法没有理解清楚，导致出错。

In [ ]:
s = 'when i was just a little girl, i ask my mother.'

In [ ]:
s.index('was')

In [ ]:
s.rindex('was')

In [ ]:
s.find('was')

In [ ]:
s.rfind('was')

程序运行结果与我们的预期不一致怎么办？

请相信计算机是对的，努力根据计算机提供的信息去寻找一个合理的解释，通常那就是真相。

这也是一种**科学方法**。

**科学方法**

1. 定义 问题
2. 观察 采集相关信息和资源
3. 建立能解释观察结果的假说
4. 通过进一步的实验来验证假说
5. 分析实验中获得的新的信息
6. 建立新的假说/改进之前的假说
7. 重复4-6步
8. 发表你的发现和假说，让同行评审

这不是做科学研究的唯一正确的方法，那还有什么方法呢？



一开始的时候是盲从，等你经验丰富到一定程度的时候，就可以更多地依赖自己的判断了。

关于少儿编程。

孩子的能力培养中最重要的三种能力：

- 观察和理解世界
- 解决问题
- 表达自己

# 树 编程课第十一课

- 分类学
- 行政区划
- 公司/企业组织架构

一个软件的价值，在于它能在多大范围内被重复使用多少次。

设计数据结构的起点是分析应用场景。

1. 定义问题域 domain
2. 设计软件表达数据结构——从未来会怎么用这个软件出发来设计软件，应用驱动，不是唯一的软件设计方法
3. 具体设计，属性、方法、功能——例如行政区划的管理

**进一步抽象**

- node 节点是树结构中的关键
- 父子关系是树节点node中的关键。
- 树具有递归特征。

tree ::= root & \[sub-tree of root's children\]

以根节点作为树的 id。

# 自己写的 tree 的测试

In [ ]:
from vor.tree_node import TreeNode

In [ ]:
root = TreeNode(name='root', data='中国', parent=None, children=None)

In [ ]:
from IPython import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
root.name
root.data
root.parent
root.children

In [ ]:
print(root.parent)

In [1]:
from vor.tree_node import TreeNode

root = TreeNode(name='root', data='中国', parent=None, children=None)

node1 = TreeNode(name='node1', data='山西', parent=root)
node2 = TreeNode(name='node2', data='湖北', parent=root)
node3 = TreeNode(name='node3', data='北京', parent=root)
node4 = TreeNode(name='node4', data='上海', parent=root)

node11 = TreeNode(name='node11', data='晋城', parent=node1)
node12 = TreeNode(name='node12', data='长治', parent=node1)

node21 = TreeNode(name='node21', data='武汉', parent=node2)
node22 = TreeNode(name='node22', data='鄂州', parent=node2)

node31 = TreeNode(name='node31', data='北京', parent=node3)

node111 = TreeNode(name='node111', data='阳城', parent=node11)
node211 = TreeNode(name='node211', data='洪山', parent=node21)
node311 = TreeNode(name='node311', data='昌平', parent=node31)

add node *node1* as child of  node *root*!
add node *node2* as child of  node *root*!
add node *node3* as child of  node *root*!
add node *node4* as child of  node *root*!
add node *node11* as child of  node *node1*!
add node *node12* as child of  node *node1*!
add node *node21* as child of  node *node2*!
add node *node22* as child of  node *node2*!
add node *node31* as child of  node *node3*!
add node *node111* as child of  node *node11*!
add node *node211* as child of  node *node21*!
add node *node311* as child of  node *node31*!


In [ ]:
root.add_child(node1)

In [ ]:
root

In [ ]:
for child in root.children:
    print (child.name, child.data)

## find parent

In [ ]:
node1.find_parent()

In [ ]:
node21.find_parent()

## find all the children

In [ ]:
a = root.find_children()

In [ ]:
root.find_children()

In [ ]:
for node in a:
    print(node.name, node.data)

In [ ]:
node1.find_children()

In [ ]:
node.children

In [ ]:
a = []

In [ ]:
a.append(node11.find_children())

In [ ]:
len(a)

## 关于树状结构节点重复性的想法

同一个父节点的子节点中，允许名字和数据都相同的节点存在吗。

不同的子节点可以数据相同，但名字必须不同。名字相当于是id，唯一标识符。比如两个人可以有相同的身高、体重、发型，但这两个人是不同的。

名字相同时不允许存在的。

In [ ]:
node1.find_children()

## 找出某个特定的 child

In [ ]:
def else_test(t):
    for i in range(t):
        if i == 10:
            return i
    else:
        return("No i")

In [ ]:
else_test(10)

In [ ]:
node1.find_child(data='晋城')

In [ ]:
root.find_child(name='山西')

In [ ]:
root.find_child(data='山西')

In [ ]:
root.find_child(data='湖北')

In [ ]:
t = root.find_child(name='node4')

In [ ]:
type(t)

In [ ]:
t[0].data

In [ ]:
name = 'node1'

In [ ]:
child_dict = root.find_children()
child_dict

In [ ]:
for child in child_dict:
    print(type(child.name))

In [ ]:
target = {}
for child in child_dict:
    if child.name == name:
        target[child.name] =  child.data

In [ ]:
target

In [ ]:
if len(target) == 0:
    print ("Sorry, no node with the name *{}*.")

In [ ]:
t.name

In [ ]:
node23 = TreeNode(name='node23', data='武汉', parent=node2)

In [ ]:
node2.data

In [ ]:
a = node2.find_child(data='武汉')

In [ ]:
a

In [ ]:
for child in child_dict:
    print(child.name)

## find_siblings

In [ ]:
node2.data

In [ ]:
t = node2.find_siblings()

In [ ]:
[i.data for i in t]

In [ ]:
node21.find_siblings()[0].data

In [ ]:
node21.find_same_generation()

In [ ]:
all_sib = node1.find_parent().find_children()
all_sib

In [ ]:
[i for i in all_sib if i != node1]

In [ ]:
root.find_siblings()[0].data

In [ ]:
node1.find_siblings()

## find_same_tier

In [ ]:
t = node11.find_same_tier(flatten=True)

In [ ]:
[item.data for item in t]

In [ ]:
root.find_same_tier()[0].data

In [ ]:
root.find_same_tier()

## find_descendants

In [ ]:
node11.children

In [ ]:
t = root.find_descendants()

In [ ]:
descendants = {}
generation = 1
descendants[generation] = root.find_children()

In [ ]:
t[3][0].name

In [ ]:
tmp = []

In [ ]:
descendants

In [ ]:
for node in descendants[generation]:
    print(type(node.find_children()))

In [ ]:
descendants[generation][0]

In [ ]:
descendants[generation][0].find_children()

In [ ]:
for node in descendants[generation]:
    if len(node.find_children()) == 0:
        continue
    tmp.append(node.find_children())
    descendants[generation + 1] = [item for sublst in tmp for item in sublst]

In [ ]:
descendants[generation + 1]

In [ ]:
descendants[generation + 1] = [item for sublst in tmp for item in sublst]

In [ ]:
descendants[generation + 1]

In [ ]:
descendants[2]

In [ ]:
[item for sublst in descendants[2] for item in sublst]

In [ ]:
node21.find_descendants()

In [ ]:
node21.data

In [ ]:
node1.find_descendants()

In [ ]:
t = root.find_descendants()
for item in t[0]:
    print(item.data)

In [ ]:
t[0]

## node_tier

In [ ]:
node111.parent.parent.parent.name

In [ ]:
root.parent

In [ ]:
print(root.parent)

In [ ]:
node111.node_tier()

In [ ]:
root.node_tier()

In [ ]:
node21.node_tier()

关于兄弟节点和同代节点之间的区别。

In [ ]:
# 兄弟节点
node11.find_siblings()

In [ ]:
# 同代节点
node11.find_same_tier()

## find_ancestors

关于祖先节点。怎样算是祖先，是只要在上一代就算，还是只有直系父母的兄弟姐妹才算。

这里认为，直系父母的兄弟姐妹才算。

In [ ]:
node11.find_ancestors()

In [ ]:
root.find_ancestors()

In [ ]:
node111.find_ancestors()

In [ ]:
node11.parent.data

In [ ]:
tmp = node11
ances = tmp.find_parent()
level = ances.find_children()
level

In [ ]:
for item in level:
    print(item.data)

In [ ]:
ances

In [ ]:
level

In [ ]:
tmp = ances
level = ances.find_children()

In [ ]:
tmp.data

In [ ]:
level

## find_path

In [ ]:
tier1 = node111.node_tier()
tier1

In [ ]:
tier2 = node21.node_tier()

In [ ]:
tier2

In [ ]:
path1 = [node111]
path2 = [node21]

In [ ]:
tmp = node111
for i in range(tier1 - tier2):
    path1.append(tmp.parent)
    tmp = tmp.parent

In [ ]:
path1

In [ ]:
tmp.node_tier() != node21.node_tier()

In [ ]:
tmp.node_tier() == node21.node_tier()

In [ ]:
t = node111.find_path(node11)
for item in t:
    print(item.data)

In [ ]:
self = node1
node = node211

path1 = [self]
path2 = [node]
tier1 = self.node_tier()
tier2 = node.node_tier()
if tier1 > tier2:
    tmp = self
    for i in range(tier1 - tier2):
        path1.append(tmp.parent)
        tmp = tmp.parent
elif tier1 < tier2:
    tmp = node
    for i in range(tier2 - tier1):
        path2.append(tmp.parent)
        tmp = tmp.parent

In [ ]:
tmp.data

In [ ]:
node21.data

In [ ]:
for i in node2.find_path(node211):
    print(i.data)

In [ ]:
for i in node111.find_path(node1):
    print(i.data)

In [ ]:
for i in node1.find_path(node2):
    print(i.data)

In [ ]:
path2[::-1]

In [ ]:
node111.find_path(node1)

In [ ]:
for i in node111.find_path(node21):
    print(i.data)

In [ ]:
node111.find_distance(node21)

## add_node

In [ ]:
node221 = TreeNode(name='node221', data='地大新校区')

In [ ]:
node22.add_child(node221)

In [ ]:
node = node22.find_children()

In [ ]:
node221

In [ ]:
node

In [ ]:
node[0].data

In [ ]:
node[1].data

In [ ]:
t = 0x15d934f5c40

In [ ]:
t

In [ ]:
bin(t)

## del_node

In [ ]:
root.find_children()

In [ ]:
node2.del_node()

In [ ]:
root.find_children()

In [ ]:
for i in root.find_children():
    print(i.data)

In [ ]:
del node2

In [ ]:
node2.find_children()[0].data

In [ ]:
node1.children

In [ ]:
root.children

In [ ]:
lst = [1, 2, 3, 4, 5]
for i in lst:
    if i == 4:
        lst.remove(i)
        
print(lst)

In [ ]:
node1.find_siblings()

In [ ]:
node2.find_ancestors()

In [ ]:
node21.find_ancestors()[0][0].data

## set parent

In [ ]:
root.find_children()

In [ ]:
node1.find_descendants()

In [ ]:
node2.find_descendants()

In [ ]:
node2.set_parent(node111)

In [ ]:
node1.find_descendants()

In [ ]:
for i in node1.find_siblings():
    print(i.data)

In [ ]:
self = node2
node = node111

In [ ]:
self.parent = node

In [ ]:
self.parent.data

In [ ]:
node.children

In [ ]:
node.add_child(self)

In [ ]:
node.children[0].data

In [ ]:
sibs = self.parent.find_children()
sibs

In [ ]:
self.parent.find_children()[0].data

In [ ]:
self.data

In [ ]:
self.parent.data

In [ ]:
node111.find_descendants()

In [ ]:
node111.find_descendants()[0][0].data

In [ ]:
print("\u2514\n\u2514\u251c")

In [ ]:
print("\u251c北京\n\u251c上海\n\u2514重庆")

## show_tree

In [2]:
root.show_tree()

└─中国
     ├─山西
     │   ├─晋城
     │   │   └─阳城
     │   └─长治
     ├─湖北
     │   ├─武汉
     │   │   └─洪山
     │   └─鄂州
     ├─北京
     │   └─北京
     │        └─昌平
     └─上海


In [3]:
node111.show_tree()

└─阳城


In [4]:
node11.show_tree()

└─晋城
     └─阳城


In [5]:
node1.show_tree()

└─山西
     ├─晋城
     │   └─阳城
     └─长治
